In [81]:
import glob
import os
import ast
import pandas as pd
import google.cloud.aiplatform as aiplatform
%reload_ext autoreload
%autoreload 2
from utils import run_llm_label_flow,load_in_data,grab_specific_tag_data_breakdown_test_and_train,chunk_dataframe_into_batches,run_llm_label_flow_gemini
from constants import query_tags,sentiment_dict
from sklearn.metrics import classification_report
pd.set_option('max_colwidth', 800)

In [2]:
# !pip3 install --upgrade --user google-cloud-aiplatform

In [7]:
from vertexai.preview.generative_models import GenerativeModel, Part
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "credentials_gpe-analytics.json"
pid = 'gpe-analytics'
aiplatform.init(project=pid)

model = GenerativeModel("gemini-pro")

# def get_chat_response(message):
# vertexai.init(project="gpe-analytics", location="us-central1")
# model = GenerativeModel("gemini-pro")
# chat = model.start_chat()
# response = chat.send_message(message)
# return response.text

# Notes



1. Note that sentiment is gathered for each UNIQUE sentiment to reduce load and then remapped to all data that has that snippet for faster processing time 

# Glasses Sentiment Analysis 

In [2]:
glasses_data = load_in_data('glasses')

##  Google Tag 

In [3]:
google_glasses_data, unique_google_glasses_data, excluded_uid_google_glasses, google_glasses_train, google_glasses_test = grab_specific_tag_data_breakdown_test_and_train(glasses_data,'google')

In [85]:
# Verification
len(google_glasses_data), len(unique_google_glasses_data), len(excluded_uid_google_glasses), len(google_glasses_data)-len(unique_google_glasses_data)

(5000, 3176, 1824, 1824)

In [55]:
# Grab labels from gemini
for x,y in zip(google_glasses_test[25:]['Snippet'],google_glasses_test[25:]['Url']):
    print(x+'\n')


@PhilRay62 @openshutter21 @VisualsbySauter Google glass 😮

@MrAwesomeSays @MKBHD Same happened to Google glass

Let’s not forget the Google glass commercial because she didn’t have to go that hard 😭

@streachybear @LiuSkin I wanna make her worse. Get AI voices, hypno programs, turn my place into a smart home where I cannot escape her watch, add screens for her to command me no matter where I go, wear those stupid google glasses from ages ago, downgrade my pc account to guest and make her admin...

and we'll continue to look at ways to bring new, innovative AR experiences across our product portfolio.” I know at least one TC staffer who is gutted by the inevitable second death of Google Glass. I won't call them out by name, but will say that I...

Google glass wouldn't have failed if it simply superimposed an anime girl into my daily life

Hard to square track record of product quality from Apple with the sheer ambition + promises made in the release video given experiences with product

In [5]:
gemini_labels = {
    "Google Glass really was ahead of it’s time": "positive",
    "Once again, Google kills Google Glass#DeathOfGoogleGlass": "negative",
    "Reminds me of this funny Google Glass parody from Rocket Jump.": "neutral",
    "Everyone is reacting to Apple's AR/VR glasses like Google Glass wasn't around 10 years ago.": "neutral",
    "$GOOGL Google Glass Is Going Away, Again": "negative",
    "Kim Hanbin said during Poseidon performance he couldn't see anything because he was wearing google glass": "neutral",
    "First google glasses. Now Apple VisionPro. 3rd time is a charm. Who will be the Tesla of AR?": "neutral",
    "...just float things in in front of you. Google Glass, Nreal Air, and Vuzix Blade 2 are examples of this. Pokémon Go is often cited as an AR app, and this often gets conflated with the fact that you play it out in the real world. That's just location-based...": "neutral",
    "RT @ZGJR When I first used Google Glass during live #surgery in 2013, becoming the first surgeon to ever do that, all I did was to use the technology differently, to augment me as surgeon and educator! The smart application of #technology empowers connectivity an…": "positive",
    "...technologies that combine the properties of augmented and virtual reality. Augmented Reality blends software with the real world by overlaying graphics over the user's surroundings, similar to Google Glass. In comparison, virtual reality includes the...": "neutral",
    "RT @KILLIaKILL google glass that makes you call your grandpa": "neutral",
    "What about Google glasses - where's it now? Was really good idea and device": "neutral",
    "Google glass runs 0 android apps. None nada. It can show you the weather and take pictures.": "negative",
    "...shows Hollywood cameramen shooting movies with iPhones. The piece ends with what could only be seen as an ominous technological development for the Society of Camera Operators: Google Glass , which has the potential to turn every single person into a...": "negative",
    "this was my original hope with Google Glass, sadly,": "negative",
    "Isn't this just google glass updated?": "neutral",
    "fair I'm just glad to see the AR scene isn't married to the bulky headsets after how google glass was drove into the ground.": "positive",
    "I had Google Glass, this looks cool. It will be interesting to see as we get closer to launch what it really is.": "neutral",
    "#Meta has released a new pair of #SmartGlasses from #RayBan. As a prior owner of #GoogleGlass, I give my initial impressions and take some sweet footage of doggies in my backyard! My Video:": "neutral",
    "If (now defunct) Delorean made trucks, Cybertrucks would be it! Definitely like the vehicles in 80's sci-fi movies, but also (slightly) reminds me of the Hummer HX that was halted when General Motors filed for bankruptcy in 2009. *All said, Google Glass was futuristic": "neutral",
    "Agree, but because of their own created hype it’s going to be their iPhone or a Google glasses moment?": "neutral",
    "...hardware divisions at Google might try to build a rival product, so they are keeping the project under wraps. Despite the challenges, it seems that Google is not giving up on augmented reality (AR) glasses altogether. Reports from South Korean outlet SBS...": "neutral",
    "...by Google Lens now. Was Lens software inside the augmented-reality glasses prototyped at Google I/0 2022? Unconfirmed, but quite possible. Google CEO Sundar Pichai opined for a world filled with augmented reality accessed without a smartphone at the...": "neutral",
    "👾 RGB gaming Chromebooks have arrived // ✨ Los Chromebooks para juegos RGB han llegado 🚨🚨 Follow 👉🏻 thedailytechh para saber las Últimas noticias Tech ! #thedailytechh #google #googleleads #googleleaks #googlenews #googletech #googletechnology #googleglass #rgbgamingpc #rgbgamingsetup #rgbgaming #rgb #googlestore #googlenest...": "neutral",
    "Iq using google glasses vs solis using oculus rift": "neutral",
    "Google glass 😮": "positive",
    "Same happened to Google glass": "negative",
    "Let’s not forget the Google glass commercial because she didn’t have to go that hard 😭": "positive",
    "I wanna make her worse. Get AI voices, hypno programs, turn my place into a smart home where I cannot escape her watch, add screens for her to command me no matter where I go, wear those stupid google glasses from ages ago, downgrade my pc account to guest and make her admin...": "negative",
    "and we'll continue to look at ways to bring new, innovative AR experiences across our product portfolio.” I know at least one TC staffer who is gutted by the inevitable second death of Google Glass. I won't call them out by name, but will say that I...": "negative",
    "Google glass wouldn't have failed if it simply superimposed an anime girl into my daily life": "positive",
    "Hard to square track record of product quality from Apple with the sheer ambition + promises made in the release video given experiences with products like it in the past (Snap Spectacles, Google Glass, lol). But if anyone can do it, it is Apple.": "neutral",
    "RT @adamrsweet_ @krishnanrohit Here's a list of the top 50 Google products that have been killed in the last decade or so: Google Reader Google Wave Google Buzz Google+ Google Glass (Explorer Edition) Google Notebooki GooglePicasa Google Talk Google Video Google Health Google Labs Google Answers Google News…": "negative",
    "Back in the Google Glass days.. We called them Glassholes.": "negative",
    "Some of us remember Google Glass *shutters*": "negative",
    "RT @stealthygeek If you thought Google Glass didn't make you look enough of a punchable dork, Apple has you covered. https://t.co/QzGUdSFzMy": "negative",
    "Just like Google Glass got the name Glass-Hole this will be given the name Pin-Hole.": "negative",
    "RT @AndroidPolice Google's future AR glasses might have a different input method to the Google Glass https://t.co/7jl8wHS5st": "neutral",
    "They used to call the early adopters of Google Glass 'glassholes'. Really can't wait for what the internet comes up for this generation 🤣": "negative",
    "RIP Google Glass: de gadget is dood, alweer https://t.co/eyTPW4SJkw https://t.co/fAresJZiNV": "negative",
    "“I promise I absolutely will not be wearing a wire, or Google Glasses while present for the anonymous jury selection. Pinky swear.”": "neutral",
    "So Apple Vision Pro is just a bulkier Google Glasses #WWDC2023": "negative",
    "...???????40.6??-47????????58%?????????????????????????????????? ... ... ?????????????????Snapchat????????Snap???????????Spectacles??????????????????????????Google Glass??????????????iPhone?????????????????????????? stratechery ??? Ben Thompson ?????????????...": "neutral",
    "Wonder if it'll go the same way as Google Glass🤔 A cool idea - and definitely will have some applications.": "neutral",
    "AVP is going to fail like the Google Glasses, unless the price is dropped and the form factor is made less awkward.": "negative",
    "RT @fabrizio_degni Another candidate for the #Google #Cemetery: the #Iris #Smartglasses Project. It was the plan for #AugmentedReality smart glasses as successor of the well know #GoogleGlasses. Reason?It seems to keep focusing on the #software than #devices. Source: https://t.co/VGgfe89sov #AR https://t.co/VipvMI9hi9": "negative",
    "Yes true I read it on my googleglasses https://t.co/CFKkAbbkAB": "neutral",
    "Google Glass": "neutral",
    "As everyone knows the biggest problem people had with Google Glass was that it was too understated so I'm glad they addressed that": "positive",
    "Can someone tell me what happened to the Google Glasses? I’m afraid the EV’s will face the same fate! BUT, love American innovation & ingenuity!! 🇺🇸🙏🏽🙏🏼": "neutral",
}


In [6]:
google_glasses_test['gemini_label'] = list(gemini_labels.values())

In [64]:
google_glasses_test.to_csv('../data/glasses_test_data/google_glasses_test.csv')

### Compare gemini labels to mine 


In [7]:
google_glasses_test = pd.read_csv('../data/glasses_test_data/google_glasses_test.csv')

In [8]:
google_glasses_test['gemini_label'] = google_glasses_test['gemini_label'].apply(lambda x: sentiment_dict[x])

In [9]:
google_glasses_test['disagreement'] = google_glasses_test.apply(lambda x: x.gemini_label!=x.true_label,axis=1)

In [10]:
print(classification_report(google_glasses_test['true_label'],google_glasses_test['gemini_label']))

              precision    recall  f1-score   support

           0       0.89      0.89      0.89        18
           1       0.88      0.88      0.88        25
           2       0.57      0.57      0.57         7

    accuracy                           0.84        50
   macro avg       0.78      0.78      0.78        50
weighted avg       0.84      0.84      0.84        50



In [100]:
google_glasses_test['llm_label'].value_counts()/len(google_glasses_test)

llm_label
1    0.50
0    0.34
2    0.16
Name: count, dtype: float64

In [76]:
google_glasses_test.loc[google_glasses_test['disagreement']==True][['Snippet','gemini_label','true_label']]

,Snippet,gemini_label,true_label
11,@ThisIsMeIn360VR What about Google glasses - where's it now? Was really good idea and device,1,2
16,@SodaScarlet fair I'm just glad to see the AR scene isn't married to the bulky headsets after how google glass was drove into the ground.,2,0
17,"@adamhlavac I had Google Glass, this looks cool. It will be interesting to see as we get closer to launch what it really is.",1,2
19,"@teslaownersSV If (now defunct) Delorean made trucks, Cybertrucks would be it! Definitely like the vehicles in 80's sci-fi movies, but also (slightly) reminds me of the Hummer HX that was halted when General Motors filed for bankruptcy in 2009. *All said, Google Glass was futuristic https://t.co/mgUfYzyI2g",1,2
26,@MrAwesomeSays @MKBHD Same happened to Google glass,0,1
27,Let’s not forget the Google glass commercial because she didn’t have to go that hard 😭,2,1
30,Google glass wouldn't have failed if it simply superimposed an anime girl into my daily life,2,0
41,So Apple Vision Pro is just a bulkier Google Glasses #WWDC2023,0,1


In [81]:
test = {
 "What about Google glasses - where's it now? Was really good idea and device": "positive",
 "fair I'm just glad to see the AR scene isn't married to the bulky headsets after how google glass was drove into the ground.": "positive",
 'I had Google Glass, this looks cool. It will be interesting to see as we get closer to launch what it really is.': "neutral",
 "If (now defunct) Delorean made trucks, Cybertrucks would be it! Definitely like the vehicles in 80's sci-fi movies, but also (slightly) reminds me of the Hummer HX that was halted when General Motors filed for bankruptcy in 2009. *All said, Google Glass was futuristic": "positive",
 'Same happened to Google glass': "negative",
 'Let’s not forget the Google glass commercial because she didn’t have to go that hard 😭': "positive",
 "Google glass wouldn't have failed if it simply superimposed an anime girl into my daily life": "positive",
 'So Apple Vision Pro is just a bulkier Google Glasses #WWDC2023': "negative"
}
values = list(test.values())

In [83]:
values = [sentiment_dict[x] for x in values]

In [118]:
values

[2, 2, 1, 2, 0, 2, 2, 0]

In [ ]:
["positive", "positive", "neutral", "neutral", "negative", "negative", "negative", "negative", "neutral"]

In [84]:
google_glasses_test.loc[google_glasses_test['disagreement']==True,'gemini_label'] = values

In [85]:
# Stick to 90% accuracy
print(classification_report(google_glasses_test['true_label'],google_glasses_test['gemini_label']))

              precision    recall  f1-score   support

           0       0.89      0.89      0.89        18
           1       0.96      0.88      0.92        25
           2       0.67      0.86      0.75         7

    accuracy                           0.88        50
   macro avg       0.84      0.88      0.85        50
weighted avg       0.89      0.88      0.88        50



In [11]:
# Confirm batch job works
google_glasses_batched_test = chunk_dataframe_into_batches(google_glasses_test,20)

In [12]:
test_dict = {uid:text for uid,text in zip(google_glasses_batched_test[0]['uids'],google_glasses_batched_test[0]['texts'])}

In [90]:
# NOTE make predictions ONLLLLLLY on unique snippets 
test_prompt = """Hi Gemini! I want you to act as an expert in sentiment! 
I am trying to get read on what the sentiment is for each post SPECIFIC to Google Glasses. 
I.e. is the sentiment, positive, negative, or neutral TOWARDS Google Glasses.
Think through your answer, and remind yourself is this SPECIFICALLY POSITIVE, NEGATIVE, or NEUTRAL towards Google Glasses, or is this tone just
positive,negative,neutral in general.
My life depends on getting the entity-specifc answer correct! 
Again when labeling the sentiment label it under the condition of its SPECIFIC sentiment towards Google Glasses
\n Return the labels in the form: {"link": sentiment",...} in the order of the posts 

Posts:"""




test_prompt_text_bison = """I want you to act as an expert in sentiment! 
I am trying to get read on what the sentiment is for each post SPECIFIC to Google Glasses. 
I.e. is the sentiment, positive, negative, or neutral TOWARDS Google Glasses.
Think through your answer, and remind yourself is this SPECIFICALLY POSITIVE, NEGATIVE, or NEUTRAL towards Google Glasses, or is this tone just
positive,negative,neutral in general.
My life depends on getting the entity-specifc answer correct! 
Again when labeling the sentiment label it under the condition of its SPECIFIC sentiment towards Google Glasses
\n Output: -positive, -negative, -neutral. 

Posts:"""

In [85]:
google_glasses_test = google_glasses_test.rename(columns={'Url':'uid','Snippet':'text'})

In [95]:
results_test_tb = run_llm_label_flow(sentiment_dict, test_prompt_text_bison, ['positive','negative','neutral'], google_glasses_test, label_name='llm_label')

100%|███████████████████████████████████████████| 50/50 [00:02<00:00, 18.61it/s]


In [98]:
print(classification_report(results_test_tb['true_label'],results_test_tb['llm_label']))

              precision    recall  f1-score   support

           0       0.68      0.94      0.79        18
           1       1.00      0.36      0.53        25
           2       0.44      1.00      0.61         7

    accuracy                           0.66        50
   macro avg       0.71      0.77      0.64        50
weighted avg       0.81      0.66      0.63        50



In [74]:
results_test = run_llm_label_flow_gemini(sentiment_dict, test_prompt, ['placeholder'], google_glasses_batched_test, label_name='llm_label')

100%|█████████████████████████████████████████████| 3/3 [00:03<00:00,  1.00s/it]


In [105]:
test_data = [k['output'] for k in results_test]

,0
0,"{""http://twitter.com/YabukiJoe517/statuses/1674515275138609155"": 'positive', ""http://twitter.com/bbsutton/statuses/1637450640845840384"": 'negative', ""http://twitter.com/PJBgamer/statuses/1665949808870277121"": 'neutral', ""http://twitter.com/ashleysmithIRL/statuses/1616083462154223616"": 'neutral', ""http://twitter.com/GetOnTrade/statuses/1636420835434184705"": 'negative', ""http://twitter.com/SunFlow1022131/statuses/1687878640565137408"": 'neutral', ""http://twitter.com/dc10023/statuses/1666012616731312129"": 'neutral', ""https://www.macworld.com/article/1792115/apples-reality-pro-headset-explained-what-do-ar-vr-and-mr-mean.html"": 'neutral', ""http://twitter.com/roxannriskin/statuses/1639306575159476239"": 'positive', ""https://hitechglitz.com/samsung-google-and-qualcomm-are-joining-forces-to-buil..."
1,"{""http://twitter.com/vvdheuvel/statuses/1722719000004567351"": ""neutral"", ""https://urecomm.com/googles-ar-projects-are-reportedly-facing-even-more-setbacks/"": ""neutral"", ""https://blog.dailyfeedmail.net/5-future-scenarios-for-google-lens/"": ""neutral"", ""https://www.instagram.com/p/Cjq20K7poHw/"": ""neutral"", ""http://twitter.com/ryuu_1320/statuses/1596380853889732608"": ""neutral"", ""http://twitter.com/Denise_W63/statuses/1690731044180283392"": ""positive"", ""http://twitter.com/KrisztianTSzucs/statuses/1665799477771247617"": ""negative"", ""http://twitter.com/samelon28/statuses/1617023474941284353"": ""positive"", ""http://twitter.com/Felhel2/statuses/1672711140982358018"": ""negative"", ""https://xtechnews.com/2023/03/16/goodbye-google-glass-we-knew-you-well/"": ""neutral"", ""http://twitter.com/NeedSumikaWife/s..."
2,"{""http://twitter.com/pabloalto/statuses/1648005736197410817"": ""neutral"", ""http://twitter.com/lakobe2/statuses/1665797441881358337"": ""negative"", ""http://yunjuu.com/info/2096208.html"": ""neutral"", ""http://twitter.com/PFriend82/statuses/1666012973498859523"": ""neutral"", ""http://twitter.com/Kirankool/statuses/1681013076924006400"": ""negative"", ""http://twitter.com/LivingInHM/statuses/1674807619179913218"": ""negative"", ""http://twitter.com/smellycommie/statuses/1657010068578086916"": ""neutral"", ""http://twitter.com/Raidmultinsetos/statuses/1655556754321620992"": ""neutral"", ""http://twitter.com/DECfromOH/statuses/1665793546362707969"": ""negative"", ""http://twitter.com/bfe1973/statuses/1706340615800144141"": ""neutral""}"


In [34]:
len(ast.literal_eval(results_test[2]['output']))

10

In [75]:
uid_dict = {}

for i in range(len(results_test)):
    try:
        uid_dict.update(ast.literal_eval(results_test[i]['output']))
    except:
        print(i,results_test[i]['output'])
# def update_dictionary(result_list):
#     results_test[0]['output']

In [27]:
len(test_labels)

48

In [63]:
sentiment_dict['sarcastic'] = 0

In [76]:
google_glasses_test['llm_label'] = google_glasses_test['Url'].apply(lambda x: sentiment_dict[uid_dict[x]])

In [77]:
print(classification_report(google_glasses_test['true_label'],google_glasses_test['llm_label']))

              precision    recall  f1-score   support

           0       0.82      0.78      0.80        18
           1       0.84      0.84      0.84        25
           2       0.50      0.57      0.53         7

    accuracy                           0.78        50
   macro avg       0.72      0.73      0.72        50
weighted avg       0.79      0.78      0.78        50



In [78]:
# Look into differences 
google_glasses_test['disagreement'] = google_glasses_test.apply(lambda x: x.true_label!=x.llm_label,axis=1)

In [79]:
google_glasses_test.loc[google_glasses_test['disagreement']==True][['Snippet','true_label','llm_label']]

,Snippet,true_label,llm_label
13,"...shows Hollywood cameramen shooting movies with iPhones. The piece ends with what could only be seen as an ominous technological development for the Society of Camera Operators: Google Glass , which has the potential to turn every single person into a...",0,1
14,"@loftypurple this was my original hope with Google Glass, sadly,",0,2
17,"@adamhlavac I had Google Glass, this looks cool. It will be interesting to see as we get closer to launch what it really is.",2,1
18,"#Meta has released a new pair of #SmartGlasses from #RayBan. As a prior owner of #GoogleGlass, I give my initial impressions and take some sweet footage of doggies in my backyard! My Video: https://t.co/2g0QjQtnFd",1,2
19,"@teslaownersSV If (now defunct) Delorean made trucks, Cybertrucks would be it! Definitely like the vehicles in 80's sci-fi movies, but also (slightly) reminds me of the Hummer HX that was halted when General Motors filed for bankruptcy in 2009. *All said, Google Glass was futuristic https://t.co/mgUfYzyI2g",2,1
26,@MrAwesomeSays @MKBHD Same happened to Google glass,1,0
27,Let’s not forget the Google glass commercial because she didn’t have to go that hard 😭,1,2
29,"and we'll continue to look at ways to bring new, innovative AR experiences across our product portfolio.” I know at least one TC staffer who is gutted by the inevitable second death of Google Glass. I won't call them out by name, but will say that I...",0,1
30,Google glass wouldn't have failed if it simply superimposed an anime girl into my daily life,0,2
41,So Apple Vision Pro is just a bulkier Google Glasses #WWDC2023,1,0
